# 04 · 新資料推論（TIC -> 機率）

In [ ]:
# 環境設定與依賴安裝（Colab）
import sys, subprocess, pkgutil, joblib, json

def pipi(*pkgs):
    """安裝套件的輔助函式"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])

# 安裝必要套件（避免 numpy 2.0 相容性問題）
print("🚀 正在安裝依賴套件...")
try:
    import numpy as np
    import lightkurve as lk
    print("✅ 基礎套件已安裝")
except Exception:
    pipi("numpy<2", "lightkurve", "astroquery", "scikit-learn", "matplotlib", "joblib")
    print("✅ 依賴套件安裝完成")

# 檢查 GPU 資訊
import torch if 'torch' in [m.name for m in pkgutil.iter_modules()] else None
if torch is not None and torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🖥️ GPU 型號: {gpu_name}")
    print(f"   記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    
    # 如果是 NVIDIA L4，提供推論優化建議
    if "L4" in gpu_name:
        print("💡 偵測到 NVIDIA L4 GPU - 支援高效能 BF16 推論")
        print("   若使用深度學習模型，建議使用：")
        print("   ```python")
        print("   with torch.autocast('cuda', dtype=torch.bfloat16):")
        print("       predictions = model(inputs)")
        print("   ```")
else:
    try:
        # 使用 nvidia-smi 檢查 GPU
        result = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], 
                              capture_output=True, text=True, check=False)
        if result.returncode == 0:
            gpu_name = result.stdout.strip()
            print(f"🖥️ GPU 型號: {gpu_name}")
            if "L4" in gpu_name:
                print("💡 偵測到 NVIDIA L4 GPU - 支援高效能 BF16 推論")
    except:
        print("⚠️ 未偵測到 GPU，將使用 CPU 進行推論")

print("\n環境設定完成！準備載入模型與推論...")

In [ ]:
import lightkurve as lk, numpy as np, json, joblib
from pathlib import Path
model_path = "/content/model/ranker.joblib"
schema_path = "/content/model/feature_schema.json"
assert Path(model_path).exists(), "請先在 03_injection_train.ipynb 產生模型"
pipe = joblib.load(model_path)
schema = json.load(open(schema_path))


In [ ]:
# 參數
TIC = "TIC 25155310"  # 可更換
sr = lk.search_lightcurve(TIC, mission="TESS", author="SPOC")
lc = sr.download().remove_nans()
flat = lc.flatten(window_length=401)
bls = flat.to_periodogram(method="bls", minimum_period=0.5, maximum_period=20)
feat = [bls.period_at_max_power.value, bls.duration_at_max_power.value, bls.max_power.value]
import numpy as np
prob = float(pipe.predict_proba(np.array(feat).reshape(1,-1))[:,1])
prob

In [ ]:
# 視覺化
ax = bls.plot(); ax.set_title("BLS Power Spectrum")
folded = flat.fold(bls.period_at_max_power, bls.transit_time_at_max_power)
ax = folded.plot(); ax.set_title("Folded Light Curve")